In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import dask
import feather, h5py, sys, pickle
#from timezonefinder import TimezoneFinder
from pytz import timezone
import pytz, os, datetime, netCDF4

In [2]:
redo_opt_output = True
run_name = 'amtk_current'
data_path = f'~/us_ego/annual_emissions/inventory_power_plants_scaled_{run_name}_noNaN_test.nc'

if redo_opt_output == False:
    # open dataset
    ds = xr.open_dataset(data_path, chunks={'time': 100})  # or chunk on spatial dims
    print(ds)

In [3]:
#run_name = 'base'
run_name = 'amtk_current'

In [4]:
if redo_opt_output == True:
    # Get power model output and merge with power plant characteristics
    gen = feather.read_dataframe(f'./outputs/gen_{run_name}.feather')
    carac = pd.read_csv(f'./inputs/inputs_gen_{run_name}.csv')
    print('data loaded')

data loaded


In [5]:
if redo_opt_output == True:
    # Clean columns name
    carac = carac.drop('Unnamed: 0', axis=1)

    # Merge
    opt_out = pd.concat((carac,gen), axis=1)
    unit_regions = opt_out['RegionName']

    # Add the missing final hour
    opt_out['2016_365_23'] = opt_out['2016_365_22'].copy()

    # Compute grid indices of all generators on NEI grid
    startlat = 20.05
    startlon = -139.95
    step = 0.1
    opt_out['idxLat'] = np.floor((opt_out['LAT'].values - startlat)/step).astype(int)
    opt_out['idxLon'] = np.floor((opt_out['LON'].values - startlon)/step).astype(int)


In [6]:
if redo_opt_output == True:
        #Check that all values are within grid limits
    if (opt_out['idxLat'] < 0).sum() + (opt_out['idxLat'] > 400).sum() +\
        (opt_out['idxLon'] < 0).sum() + (opt_out['idxLon'] > 900).sum() != 0:
        raise ValueError('Some generators are outside grid limits')

In [7]:
if redo_opt_output == True:
    base_load = pd.read_csv('~/us_ego/inputs/inputs_load.csv') # MWh
    amtk_load = pd.read_csv(f'~/us_ego/inputs/inputs_load_{run_name}.csv') # MWh

    del_load = amtk_load['demandLoad'] - base_load['demandLoad'] # load attributable to Amtrak

    mask = del_load != 0
    del_load = del_load[mask]
    range_idx = del_load.index

    # open existing file; amtrak load + base load grid emissions
    path='~/us_ego/annual_emissions/'
    name =f'inventory_power_plants_{run_name}.nc'
    base_name = f'inventory_power_plants_base.nc'
    amtrak_grid_emissions = xr.open_dataset(path+name, engine="netcdf4")
    base_grid_emissions = xr.open_dataset(path+base_name, engine="netcdf4")

    # Using Dask for large arrays
    amtrak_grid_chunked = amtrak_grid_emissions.chunk({"time": 100})  
    base_grid_chunked = base_grid_emissions.chunk({"time": 100})  

    amtrak_grid = amtrak_grid_chunked.fillna(0)
    base_grid = base_grid_chunked.fillna(0)

    #print('Base NOx emissions:',(np.sum(base_grid["NO"].values)+np.sum(base_grid["NO2"].values))*3600*123210000/1000/1000,'Tg/yr') # Tg/yr
    #print('Amtrak+base NOx emissions:',(np.sum(amtrak_grid['NO'].values)+np.sum(amtrak_grid['NO2'].values))*3600*123210000/1000/1000,'Tg/yr') # Tg/yr
    phi_region = []
    for ii in range_idx:
        if amtk_load['r'][ii] not in phi_region:
            phi_region.append(amtk_load['r'][ii])

    hours = amtk_load['t'].unique()

    # initialize array of hourly scaling values
    phi_array = np.zeros((len(hours),len(phi_region)))
    kk = 0 # region index
    jj = 0 # hour index

    for kk in range(len(phi_region)):
        print(phi_region[kk],'...')
        amtk_load_hour = amtk_load.loc[(amtk_load['r']==phi_region[kk]),'demandLoad'].values
        base_load_hour = base_load.loc[(base_load['r']==phi_region[kk]),'demandLoad'].values
        phi_array[:,kk] = np.divide(np.subtract(amtk_load_hour,base_load_hour),amtk_load_hour) # calculates the fraction of grid power used by Amtrak at each hour in each region
        print('...done!')

    phi_array = np.vstack([phi_array, phi_array[-1]])

NENG_CT ...
...done!
NENGREST ...
...done!
NY_Z_GI ...
...done!
NY_Z_J ...
...done!
NY_Z_K ...
...done!
PJM_Dom ...
...done!
PJM_EMAC ...
...done!
PJM_SMAC ...
...done!
PJM_WMAC ...
...done!


In [8]:
if redo_opt_output == True:
    print(phi_array[8758,:])
    print(phi_array[8759,:])

[1.69199913e-03 5.56362227e-04 2.81928662e-04 2.64090167e-04
 2.20526192e-04 2.35729394e-05 2.12151336e-03 1.31448392e-03
 2.29355094e-06]
[1.69199913e-03 5.56362227e-04 2.81928662e-04 2.64090167e-04
 2.20526192e-04 2.35729394e-05 2.12151336e-03 1.31448392e-03
 2.29355094e-06]


In [9]:
if redo_opt_output == True:
    print(len(phi_array))
    print(phi_region)

8760
['NENG_CT', 'NENGREST', 'NY_Z_GI', 'NY_Z_J', 'NY_Z_K', 'PJM_Dom', 'PJM_EMAC', 'PJM_SMAC', 'PJM_WMAC']


In [10]:
if redo_opt_output == True:
    #print(amtk_load['r'].unique())
    print(amtk_load[amtk_load['t']==1])

        Unnamed: 0         r  t  demandLoad
0                0  ERC_REST  1     29087.0
8759          8759  ERC_WEST  1      2655.0
17518        17518      FRCC  1     17709.0
26277        26277  MAP_WAUE  1      2551.0
35036        35036    MIS_IA  1      2332.0
...            ...       ... ..         ...
490504      490504  WECC_SCE  1     10605.0
499263      499263   WECC_SF  1       542.0
508022      508022  WECC_SNV  1      2196.0
516781      516781   WECC_UT  1      4050.0
525540      525540   WECC_WY  1      1356.0

[61 rows x 4 columns]


In [62]:
def process_emissions(species_abbrev, NO = False, NO2 = False):
    """
    Process emissions of NO, NO2, SO2, CH4 and CO2 by multiplying the emissions factor times the generation (output in kg/s)
    initial data in: $MWH/3600sec -> MW/s -> * kg/MW -> kg/s$
    """
    emis_df = pd.DataFrame(data=opt_out[f'PL{species_abbrev}RTA'].values.reshape(len(opt_out.index),1) * opt_out[[f'2016_{day}_{hour}' for day in range(1,366) for hour in range(24)]] / 3600)
    print(emis_df)
    emis_df['idxLat'] = opt_out['idxLat'].astype(int)
    emis_df['idxLon'] = opt_out['idxLon'].astype(int)
    emis_df['StateName'] = opt_out['StateName']
    if NO == True:
        mult = 0.8544304 # NO/NOx as estimated from NEI2011 inventory
    elif NO2 == True:
        mult = 1 - 0.8544304 # NO2/NOx as estimated from NEI2011 inventory
    else:
        mult = 1 # multiply by one to keep the same value if not NO or NO2

    for day in range(1, 366):
        for hour in range(24):
            jj = (day-1)*24 + hour 
            col_name = f'2016_{day}_{hour}'
            for region in unit_regions.unique():
                if region in phi_region:
                    unit_indexes = unit_regions[unit_regions == region].index
                    phi_region_index = phi_region.index(region)
                    #print(f'Grid load in {phi_region[phi_region_index]}')
                    #if override_phi == True:
                    #emis_df.loc[unit_indexes, col_name] *= mult
                    #else:
                    phi = mult*phi_array[jj,phi_region_index]
                    #print(mult,phi_array[jj,phi_region_index],phi)
                    emis_df.loc[unit_indexes, col_name] *= phi
        
                else:
                    unit_indexes = unit_regions[unit_regions == region].index
                    emis_df.loc[unit_indexes, col_name] = 0
            #print(emis_df[col_name])
            
            #print(unit_regions[unit_regions.isin(phi_region)].index)
            #unit_indexes = np.asarray(unit_regions[unit_regions.isin(phi_region)].index)
            #print(emis_df[col_name].iloc[unit_indexes])
            
            #unit_indexes = np.asarray(unit_regions[unit_regions=='NENGREST'].index)
            #print(len(emis_df[col_name].iloc[unit_indexes]))
    return(emis_df)

In [63]:
print(unit_regions)

0        ERC_FRNT
1        ERC_FRNT
2        ERC_FRNT
3        ERC_FRNT
4        ERC_GWAY
           ...   
16888    WECC_PNW
16889    MIS_WUMS
16890    MIS_WUMS
16891    MIS_WUMS
16892      PJM_AP
Name: RegionName, Length: 16893, dtype: object


In [64]:
cell_size = (11.1*1000)**2 # m^2 # corresponds to 0.1° cell resolution

In [65]:
if redo_opt_output == True:
    test_str = 'NENGREST'
    print(phi_region.index(test_str))

1


In [66]:
if redo_opt_output == True:
    # Process emissions for each species
    no = process_emissions(species_abbrev = 'NOX', NO = True, NO2 = False)
    print('no processed')

    so2 = process_emissions(species_abbrev = 'SO2')
    print('so2 processed')
    
    no2 = process_emissions(species_abbrev = 'NOX', NO = False, NO2 = True)
    print('no2 processed')

       2016_1_0  2016_1_1  2016_1_2  2016_1_3  2016_1_4  2016_1_5  2016_1_6  \
0      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
...         ...       ...       ...       ...       ...       ...       ...   
16888  0.386959  0.208666  0.140663  0.386959  0.000000  0.000000  0.218033   
16889  0.000000  0.000000  0.000000  0.000000  0.028481  0.035516  0.088073   
16890  0.088495  0.115575  0.115575  0.024119  0.115575  0.115575  0.000000   
16891  0.130307  0.130307  0.130307  0.130307  0.130307  0.130307  0.130307   
16892       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

       2016_1_7  2016_1_8  2016_1_9  ...  2016_365_

In [67]:
if redo_opt_output == True:
    # reshape NO
    print(np.shape(no))
    no_new = no.iloc[:, :8760].fillna(0)
    no_new = no_new[(no_new != 0).any(axis=1)]
    print(np.shape(no_new))

(16893, 8763)
(796, 8760)


In [74]:
if redo_opt_output == True:
    # sum NO
    emis_tot_NO = np.sum(no_new.values)*3600/1000 # Mg/yr, sum cell values
    print('NO annual emissions:',emis_tot_NO, 'Mg/yr')
else:
    emis_tot_NO = np.sum(ds['NO'].values)*cell_size/1000 # Mg/yr
    print('NO annual emissions:',emis_tot_NO, 'Mg/yr')

NO annual emissions: 319.997073379385 Mg/yr


In [69]:
if redo_opt_output == True:
    # reshape NO2
    print(np.shape(no2))
    no2_new = no2.iloc[:, :8760].fillna(0)
    no2_new = no2_new[(no2_new != 0).any(axis=1)]
    no2_new = no2_new
    print(np.shape(no2_new))

(16893, 8763)
(796, 8760)


In [75]:
if redo_opt_output == True:
    # reshape NO2
    emis_tot_NO2 = np.sum(no2_new.values)*3600/1000 # Mg/yr, sum cell values
    print('NO2 annual emissions:',emis_tot_NO2, 'Mg/yr')
else:
    emis_tot_NO2 = np.sum(ds['NO2'].values)*cell_size/1000 # Mg/yr
    print('NO2 annual emissions:',emis_tot_NO2, 'Mg/yr')

NO2 annual emissions: 54.518011031685816 Mg/yr


In [71]:
if redo_opt_output == True:
    # reshape SO2
    print(np.shape(so2))

    so2_new = so2.iloc[:, :8760].fillna(0)
    so2_new = so2_new[(so2_new != 0).any(axis=1)]
    print(np.shape(so2_new))

(16893, 8763)
(795, 8760)


In [76]:
if redo_opt_output == True:
    # sum SO2
    emis_tot_SO2 = np.sum(so2_new.values)*3600/1000 # Mg/yr, sum cell values
    print('SO2 annual emissions:',emis_tot_SO2, 'Mg/yr')
else:
    emis_tot_SO2 = np.sum(ds['SO2'].values)*cell_size/1000 # Mg/yr
    print('SO2 annual emissions:',emis_tot_SO2, 'Mg/yr')

SO2 annual emissions: 108.88799167475419 Mg/yr


In [73]:
# store values from previous run
# NO  = 746 Mg/yr
# NO2 = 109 Mg/yr
# SO2 = 478 Mg/yr